In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [ ]:
columns_to_keep_games_stats = ["PlayerId", "CreatedAt", "UpdatedAt", "Map",  "Kills", "Deaths", "Assists", "Headshots", "FirstHalfScore", "SecondHalfScore",'QuadroKills', 'TripleKills', 'PentaKills',"Headshots", "Headshots%","K/rRatio", "K/dRatio", 'Mvps', "OvertimeScore", "Rounds", "Score", "Result"]

df = pd.read_pickle("./dataCompressed.pkl", "zip")[columns_to_keep_games_stats]


In [ ]:
type_casting = {"PlayerId": str, "CreatedAt": "datetime64[ns]", "UpdatedAt":  "datetime64[ns]", "Kills": int, "Deaths": int, "Assists": int, "Headshots": int, "FirstHalfScore":int, "SecondHalfScore": int, "QuadroKills": int,  "TripleKills": int, "PentaKills": int, "Mvps": int, "OvertimeScore": int, "Rounds": int, "Result": int}
df = df.astype(type_casting, errors = "ignore")


In [ ]:
df.dropna(how = "any", inplace=True)
df["Result"] = df["Result"].astype(int).astype(bool)